# Data Frame Federated Statistics 

In this example, we will show how to generate federated statistics for data that can be represented as Pandas Data Frame. 

## Setup NVFLARE

follow the [Getting_Started](https://nvflare.readthedocs.io/en/main/getting_started.html) to setup virtual environment and install NVFLARE

You can also follow this [notebook](../../../nvflare_setup.ipynb) Notebook to setup.

Just to quick recap the nvflare install and POC installation steps in the followings

**NVFLARE Installation**

In [ ]:
!python -m pip install nvflare

In [ ]:
! nvflare -V

## Install requirements
lets first install required packages.


In [ ]:
!pip install -r df_stats/requirements.txt


## Prepare data

In this example, we are using UCI (University of California, Irwin) [adult dataset](https://archive.ics.uci.edu/ml/datasets/adult)
The original dataset has already contains "training" and "test" datasets. Here we simply assume that "training" and test data sets are belong to different clients.
so we assigned the training data and test data into two clients.
 
Now we use data utility to download UCI datasets to separate client package directory to /tmp/nvflare/data/ directory



In [ ]:
!df_stats/prepare_data.sh


## Run job in FL Simulator

With FL simulator, we can just run the example with CLI command 



In [ ]:
! nvflare simulator df_stats/jobs/df_stats -w /tmp/nvflare/df_stats -n 2 -t 2



The results are stored in workspace "/tmp/nvflare"
```
/tmp/nvflare/df_stats/simulate_job/statistics/adults_stats.json
```

In [ ]:
cat /tmp/nvflare/df_stats/simulate_job/statistics/adults_stats.json

## Visualization
We can visualize the results easly via the visualizaiton notebook. Before we do that, we need to copy the data to the notebook directory 


In [ ]:
! cp /tmp/nvflare/df_stats/simulate_job/statistics/adults_stats.json df_stats/demo/.

now we can visualize via the [visualization notebook](df_stats/demo/visualization.ipynb)

We are not quite done yet. What if you prefer to use python API instead CLI to run jobs. Lets do that in this section

## Run Job using Simulator API
This should be the same as running in command CLI via nvflare simulator

In [ ]:
from nvflare.private.fed.app.simulator.simulator_runner import SimulatorRunner
runner = SimulatorRunner(job_folder="df_stats/jobs/df_stats", workspace="/tmp/nvflare/df_stats", n_clients = 2, threads=2)
runner.run()

## Run Job using FLARE API in POC mode

So far, we having using Simulator to simulate the federated job run. With [FLARE API](../../../tutorials/flare_api.ipynb) , you can directly interact with NVFLARE system in production or POC mode
In such cases, we have to first setup and deploy the federated system. Since we are running on local machine, we will minic this deploy via Proof of Conccept mode.
Please refer this section to see how to [setup POC mode](../../../tutorials/setup_poc.ipynb) and [detailed POC commands](https://nvflare.readthedocs.io/en/latest/user_guide/poc_command.html#poc-command)

For now, we assume the NVFLARE system is already running in POC mode started with the following commands

```
nvflare poc --prepare -n 2 

nvflare poc --start -ex admin
```

You double check if the flare is running with the following command from **terminal**



In [ ]:
! echo y | nvflare poc --prepare -n 2 

In [ ]:
ls -al  /tmp/nvflare/poc/admin/transfer

In [ ]:
%%bash --bg  

nvflare poc --start -ex admin
echo "sleep for few seconds to let system start"
sleep 3

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(workspace, "admin")
sess = new_insecure_session(admin_dir)
sys_info = sess.get_system_info()

print(f"Server info:\n{sys_info.server_info}")
print("\nclient_info")
for client in sys_info.client_info:
    print(client)

repeat above step until you see Clients (site-1, site-2) are connected. 
Once they are connected, then we are ready to go

## Utilities

**Monitoring Job**

You can choose your monitoring output, here is one function to display the job information 

In [ ]:
import json
from nvflare.fuel.flare_api.flare_api import Session

def status_monitor_cb(
        session: Session, job_id: str, job_meta, *cb_args, **cb_kwargs
    ) -> bool:
    if job_meta["status"] == "RUNNING":
        if cb_kwargs["cb_run_counter"]["count"] < 3 or cb_kwargs["cb_run_counter"]["count"]%15 == 0:
            print(job_meta)            
        else: 
            print(".", end="")
    else:
        print("\n" + str(job_meta))
    
    cb_kwargs["cb_run_counter"]["count"] += 1
    return True



def format_json( data: dict): 
    print(json.dumps(data, sort_keys=True, indent=4,separators=(',', ': ')))


### submit Job

In [ ]:
examples_dir = os.path.join(admin_dir, "transfer")
job_folder = os.path.join(examples_dir, "advanced/federated-statistics/df_stats/jobs/df_stats")
job_id = sess.submit_job(job_folder)
print(job_id + " was submitted")

### Monitoring Job

You can choose your monitoring output, here is one function to display the job information 

In [ ]:
from nvflare.fuel.flare_api.flare_api import Session
def status_monitor_cb(
        session: Session, job_id: str, job_meta, *cb_args, **cb_kwargs
    ) -> bool:
    if job_meta["status"] == "RUNNING":
        if cb_kwargs["cb_run_counter"]["count"] < 3 or cb_kwargs["cb_run_counter"]["count"]%10 == 0:
            print(job_meta)            
        else: 
            print(".", end="")
    else:
        print("\n" + str(job_meta))
    
    cb_kwargs["cb_run_counter"]["count"] += 1
    return True


In [ ]:
sess.get_job_meta(job_id)

In [ ]:
sess.monitor_job(job_id, cb=status_monitor_cb, cb_run_counter={"count":0})

In [ ]:
import json

def format_json( data: dict): 
    print(json.dumps(data, sort_keys=True, indent=4,separators=(',', ': ')))


list_jobs_output = sess.list_jobs()
print( format_json(list_jobs_output))

In [ ]:
list_jobs_output_detailed = sess.list_jobs(detailed=True)
print(format_json(list_jobs_output_detailed))

### Download the result from FL Server

In [ ]:
result_dir = sess.download_job_result(job_id)


In [ ]:
! tree {result_dir}

Now we can copy the adults_stats.json to the demo folder for visualization

In [ ]:
! cp  {result_dir}/workspace/statistics/adults_stats.json df_stats/demo/.

Now we can visualize via the [visualization notebook](df_stats/demo/visualization.ipynb) as before


### Stop NVFLARE and Cleanup
If you like to stop the NVFLARE system, clean up the temp folders and workspace, we need some clean up
* stop nvflare
* remove downloaded result folder 
* clean up POC workspace

In [ ]:
! nvflare poc --stop

In [ ]:
! rm -r {result_dir}

In [ ]:
!nvflare poc --clean

## We are done !
Congratulations, you just completed the federated stats calulation with data represented by data frame
